In [1]:
import yaml
import torch
import numpy as np
from tqdm import tqdm
from model.sem import Sem
from model.ensrec import EnsRec
from torch.utils.data import DataLoader
from data import Data, SeqBPRDataset

In [2]:
with open("config/bert_config.yaml", 'r', encoding='utf-8') as f:
    args = yaml.unsafe_load(f)
data = Data(args['data'])

>>>> 数据加载完成: 15520 条交互, 6081 个用户, 1682 个物品
>>>> 基模型的预测结果加载完成: (15520, 7, 102)
                               user        item  rating      timestamp  \
0      AE22HJCBUA3OCUGL7LW4YHHANUWQ  B00005NINU     5.0  1531232972880   
1      AE22HJCBUA3OCUGL7LW4YHHANUWQ  B0089FUF6W     5.0  1531233214136   
2      AE22S7QZPM25SREWTRRTAADSFCUQ  B002PXVZAY     3.0  1388180378000   
3      AE22S7QZPM25SREWTRRTAADSFCUQ  B002PXVZGI     5.0  1388180581000   
4      AE22S7QZPM25SREWTRRTAADSFCUQ  B002CT51EM     4.0  1388180704000   
...                             ...         ...     ...            ...   
15515  AHZZN3RBWHMF3T6ED7DUWGDSK7BQ  B00007KXPH     5.0  1257339812000   
15516  AHZZN3RBWHMF3T6ED7DUWGDSK7BQ  B000EU1H3U     5.0  1257339967000   
15517  AHZZOVULYZC5L7SFEZXTKIS5CUZQ  B00283LGQY     4.0  1364993187000   
15518  AHZZOVULYZC5L7SFEZXTKIS5CUZQ  B000I0RKX2     5.0  1371734900000   
15519  AHZZOVULYZC5L7SFEZXTKIS5CUZQ  B001E547FE     5.0  1371734919000   

       user_id  item_id  
0      

>>>> 采样负样本:   0%|          | 0/6081 [00:00<?, ?it/s]


AssertionError: 

In [6]:
import pandas as pd

data = pd.read_csv("D:/Code/graduation_design/data/magazine/interaction.dat", sep='::', header=None, engine='python')
data.columns = ['user', 'item', 'rating', 'timestamp']

data = data[data['rating'] > 2]
user_counts = data['user'].value_counts()
item_counts = data['item'].value_counts()

users_to_keep = user_counts[user_counts >= 2].index
items_to_keep = item_counts[item_counts >= 2].index

data = data[data['user'].isin(users_to_keep)]
data = data[data['item'].isin(items_to_keep)]

data = data.sort_values(['user', 'timestamp'])
data = data.reset_index(drop=True)
data[:10]

,user,item,rating,timestamp
0,AE22HJCBUA3OCUGL7LW4YHHANUWQ,B00005NINU,5.0,1531232972880
1,AE22HJCBUA3OCUGL7LW4YHHANUWQ,B0089FUF6W,5.0,1531233214136
2,AE22S7QZPM25SREWTRRTAADSFCUQ,B002PXVZAY,3.0,1388180378000
3,AE22S7QZPM25SREWTRRTAADSFCUQ,B002PXVZGI,5.0,1388180581000
4,AE22S7QZPM25SREWTRRTAADSFCUQ,B002CT51EM,4.0,1388180704000
5,AE22S7QZPM25SREWTRRTAADSFCUQ,B000066B5H,5.0,1388180971000
6,AE23RS7H4JYT4OEEKGHSKNOLMUIA,B0023EW6FS,5.0,1321972965000
7,AE23RS7H4JYT4OEEKGHSKNOLMUIA,B0023EW6FS,5.0,1321972965000
8,AE23XZF2JZLER42SHFHB6BSE5IIA,B00006KFK1,5.0,1416238055000
9,AE23XZF2JZLER42SHFHB6BSE5IIA,B0045FEHCS,5.0,1487934273000


In [ ]:
with open("config/bert_config.yaml", 'r', encoding='utf-8') as f:
    args = yaml.unsafe_load(f)
data = Data(args['data'])
train_samples = np.load('datasets/new_train_samples.npy', allow_pickle=True)
test_samples = np.load('datasets/new_test_samples.npy', allow_pickle=True)
train_dataset = SeqBPRDataset(train_samples, args['data']['device'])
test_dataset = SeqBPRDataset(test_samples, args['data']['device'], is_test=True)
train_loader = DataLoader(train_dataset, batch_size=args['batch_size'], shuffle=True)

In [ ]:
# 查看用户数量
n_users = len(data['user'].unique())
print(f"用户数量: {n_users}")

In [ ]:
# 用户和物品ID映射（如果原始ID不是连续整数）
self._create_id_mappings()

# 构建用户交互字典
self.user_interacted_item_ids = defaultdict(list)
for row in self.data.itertuples():
    self.user_interacted_item_ids[row.user_id].append(row.item_id)

# 所有物品集合
self.all_item_ids = set(self.data['item_id'].unique())

print(f">>>> 数据加载完成: {len(self.data)} 条交互, {self.n_user} 个用户, {self.n_item} 个物品")

In [8]:
test_loader = DataLoader(test_dataset, batch_size=args['batch_size'], shuffle=False)

## 评估指标

- nDCG

In [4]:
def nDCG(rec_items, test_set):
    DCG = lambda x: np.sum(x / np.log(np.arange(2, len(x) + 2)))
    def get_implict_matrix(rec_items, test_set):
        rel_matrix = [[0] * rec_items.shape[1] for _ in range(rec_items.shape[0])]
        for user in range(len(test_set)):
            for index, item in enumerate(rec_items[user]):
                if item in test_set[user]:
                    rel_matrix[user][index] = 1
        return np.array(rel_matrix)
    rel_matrix = get_implict_matrix(rec_items, test_set)
    ndcgs = []
    for user in range(len(test_set)):
        rels = rel_matrix[user]
        dcg = DCG(rels)
        idcg = DCG(sorted(rels, reverse=True))
        ndcg = dcg / idcg if idcg != 0 else 0
        ndcgs.append(ndcg)
    return ndcgs

- map

In [ ]:
def map(recommended_items, interacted_items):
    interacted_set = set(interacted_items)
    
    hits, precisions = [], []
    relevant_count = 0
    for i, item in enumerate(recommended_items):
        position = i + 1  # 位置从1开始计数

        is_relevant = item in interacted_set
        hits.append(1 if is_relevant else 0)
        if is_relevant:
            relevant_count += 1
            precision_at_k = relevant_count / position
            precisions.append(precision_at_k)
    
    if not precisions:
        return 0.0
    return sum(precisions) / len(precisions)

- mrr

In [22]:
def mrr(results, relevant_docs):
    relevant_set = set(relevant_docs)
    rank = 0
    for i, doc_id in enumerate(results):
        if doc_id in relevant_set:
            rank = i + 1  # 排名从1开始
            break
    if rank > 0:
        reciprocal_rank = 1.0 / rank
        return reciprocal_rank
    else:
        return 0.0

In [7]:
def auc1(y_prob, y_label):
    n = len(y_prob)
    pos_prob = []
    neg_prob = []
    for i in range(n):
        if y_label[i]==1:
            pos_prob.append(y_prob[i])
        elif y_label[i]==0:
            neg_prob.append(y_prob[i])
    # 正样本预测概率->负样本预测概率的占比
    count = 0
    for p in pos_prob:
        for n in neg_prob:
            if p>n:
                count += 1
            elif p==n:
                count += 0.5
    return count/(len(pos_prob)*len(neg_prob))

In [7]:
def infer(model, topk, metric):
    with torch.no_grad():
        metrics = []
        for batch in tqdm(test_loader, desc="计算测试集指标"):
            all_scores = model(batch, is_test=True)
            scores, indices = torch.topk(all_scores, topk)

            for i in range(len(batch['user_id'])):
                user_id = batch['user_id'][i].item()
                pos_item = batch['pos_item'][i].item()

                true_item_ids = data.user_interacted_item_ids[user_id]
                true_item_ids = true_item_ids[true_item_ids.index(data.item_to_id[pos_item]) + 1:]

                predicted_item_ids = np.array([indices[i].cpu().numpy().tolist()])
                if metric == 'map':
                    score = map(predicted_item_ids[0], true_item_ids)
                elif metric == 'ndcg':
                    score = nDCG(np.array(predicted_item_ids), [true_item_ids])
                elif metric == 'mrr':
                    score = mrr(predicted_item_ids[0], true_item_ids)
                metrics.append(score)
    return np.mean(metrics)

## EnsRec

In [ ]:
ensrec = EnsRec(args['model'], args['data'], data.n_user)

In [ ]:
ckpt = torch.load(f"ckpt/qwen_0.3973.pth")
# filtered_ckpt = {k: v for k, v in ckpt.items() if not k.startswith('item_tower.cex')}
ensrec.load_state_dict(ckpt, strict=False)
ensrec.eval()

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=args['batch_size'], shuffle=False)
infer(ensrec, 10, 'ndcg')

## SEM

In [ ]:
sem = Sem(args['model'], args['data'], data.n_user, 3952)
ckpt = torch.load(f"../bpr/ckpt_sem/sem_epoch3.pth")
sem.load_state_dict(ckpt, strict=False)
sem.eval()

In [ ]:
infer(sem, 10, 'mrr')

## 其他集成方法

### CombSum

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
metrics = []
for batch in tqdm(test_loader, desc="计算测试集指标"):
    all_scores = batch['all_item_scores'].squeeze(0).sum(dim=0, keepdim=True)
    scores, indices = torch.topk(all_scores, 10)

    for i in range(len(batch['user_id'])):
        user_id = batch['user_id'][i].item()
        pos_item = batch['pos_item'][i].item()

        true_item_ids = data.user_interacted_item_ids[user_id]
        true_item_ids = true_item_ids[true_item_ids.index(data.item_to_id[pos_item]) + 1:]

        predicted_item_ids = np.array([indices[i].cpu().numpy().tolist()])
        score = nDCG(np.array(predicted_item_ids), [true_item_ids])
        metrics.append(score)
np.mean(metrics)

### CombMNZ

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
metrics = []
for batch in tqdm(test_loader, desc="计算测试集指标"):
    all_scores = batch['all_item_scores'].squeeze(0).sum(dim=0, keepdim=True)
    item_ids_np = batch['base_model_preds'].cpu().numpy().reshape(-1)
    item_indices = torch.tensor(np.array([data.item_to_id[id] for id in item_ids_np]), device="cuda")
    all_item_counts = torch.bincount(item_indices, minlength=data.n_item)
    all_scores = all_scores * all_item_counts.unsqueeze(0)
    scores, indices = torch.topk(all_scores, 10)

    user_id = batch['user_id'].item()
    pos_item = batch['pos_item'].item()

    true_item_ids = data.user_interacted_item_ids[user_id]
    true_item_ids = true_item_ids[true_item_ids.index(data.item_to_id[pos_item]) + 1:]

    predicted_item_ids = np.array([indices[0].cpu().numpy().tolist()])
    score = nDCG(np.array(predicted_item_ids), [true_item_ids])
    metrics.append(score)
np.mean(metrics)

### ​CombANZ

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
metrics = []
for batch in tqdm(test_loader, desc="计算测试集指标"):
    all_scores = batch['all_item_scores'].squeeze(0).sum(dim=0, keepdim=True)
    item_ids_np = batch['base_model_preds'].cpu().numpy().reshape(-1)
    item_indices = torch.tensor(np.array([data.item_to_id[id] for id in item_ids_np]), device="cuda")
    all_item_counts = torch.bincount(item_indices, minlength=data.n_item)
    # 将计数为0的位置替换为无穷大
    all_item_counts[all_item_counts == 0] = 1_000_000_000
    all_scores = all_scores / all_item_counts.unsqueeze(0)
    scores, indices = torch.topk(all_scores, 10)

    user_id = batch['user_id'].item()
    pos_item = batch['pos_item'].item()

    true_item_ids = data.user_interacted_item_ids[user_id]
    true_item_ids = true_item_ids[true_item_ids.index(data.item_to_id[pos_item]) + 1:]

    predicted_item_ids = np.array([indices[0].cpu().numpy().tolist()])
    score = nDCG(np.array(predicted_item_ids), [true_item_ids])
    metrics.append(score)
np.mean(metrics)

## 基模型预测

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

base_model_results = {}
for base_model in args['data']['base_model']:
    model = np.load(args['data']['base_model_path'] + f"/{base_model}.npy")
    ndcg_scores = []
    phar = tqdm(test_loader, desc=f"计算{base_model}测试集指标...")
    for batch in phar:
        user_id = batch['user_id'].item()
        pos_item = batch['pos_item'].item()

        interaction_idx = data.get_interaction_index(user_id, data.item_to_id[pos_item])
        assert interaction_idx != -1

        predicted_item_ids = model[interaction_idx][2:2+10]
        true_item_ids = data.user_interacted_item_ids[user_id]
        true_item_ids = true_item_ids[true_item_ids.index(data.item_to_id[pos_item]) + 1:]

        predicted_items = np.array([predicted_item_ids])
        ndcg = nDCG(np.array(predicted_items), [true_item_ids])
        ndcg_scores.append(ndcg)

        phar.set_postfix(ndcg=ndcg)
    print(f"{base_model}: {np.mean(ndcg_scores)}")